# Grabbids (grabbit) :  a few simple examples

In [4]:
import json
import os
import os.path as osp
from pprint import pprint

## This is the config file: 

In [5]:
config_file = os.path.abspath('../grabbit/tests/specs/test.json')
config = json.load(open(config_file))

### what's in the config file ?

In [7]:
!cat ../grabbit/tests/specs/test.json

{
  "entities": [
    {
      "name": "subject",
      "pattern": "sub-(\\d+)",
      "directory": "{{root}}/{subject}"
    },
    {
      "name": "session",
      "pattern": "ses-0*(\\d+)",
      "mandatory": false,
      "directory": "{{root}}/{subject}/{session}",
      "missing_value": "ses-1"
    },
    {
      "name": "run",
      "pattern": "run-0*(\\d+)"
    },
    {
      "name": "type",
      "pattern": ".*_(.*?)\\."
    },
    {
      "name": "task",
      "pattern": "task-(.*?)-"
    },
    {
      "name": "acquisition",
      "pattern": "acq-(.*?)-"
    },
    {
      "name": "bval",
      "pattern": "(.*\\.bval)"
    }
  ]
}

The config file has two components:
* a list of entities, each with a name and a pattern (a regex). Entities are things you will eventually like to grab.
* a hierarchy that tells you how these entities relate to each others on the disk.

In [8]:
import grabbit as gb
%load_ext autoreload
%autoreload 2

In [36]:
layout = gb.Layout('../grabbit/tests/data/7t_trt/', absolute_paths=False, config=config_file)

In [37]:
# This layout has entities and files: the layout.files is a dictionary, 
# which keys are the files in you root 

list(layout.files.keys())[:10]

['../grabbit/tests/data/7t_trt/sub-04/ses-2/sub-04_ses-2_scans.tsv',
 '../grabbit/tests/data/7t_trt/sub-10/ses-1/fmap/sub-10_ses-1_run-2_magnitude2.nii.gz',
 '../grabbit/tests/data/7t_trt/sub-06/ses-1/func/sub-06_ses-1_task-rest_acq-fullbrain_run-1_physio.tsv.gz',
 '../grabbit/tests/data/7t_trt/sub-09/ses-1/fmap/sub-09_ses-1_run-1_magnitude1.nii.gz',
 '../grabbit/tests/data/7t_trt/sub-06/ses-1/fmap/sub-06_ses-1_run-1_magnitude2.nii.gz',
 '../grabbit/tests/data/7t_trt/sub-06/ses-2/func/sub-06_ses-2_task-rest_acq-fullbrain_run-1_physio.tsv.gz',
 '../grabbit/tests/data/7t_trt/sub-06/ses-1/func/sub-06_ses-1_task-rest_acq-prefrontal_bold.nii.gz',
 '../grabbit/tests/data/7t_trt/sub-04/ses-2/fmap/sub-04_ses-2_run-1_phasediff.json',
 '../grabbit/tests/data/7t_trt/sub-08/ses-1/anat/sub-08_ses-1_T1map.nii.gz',
 '../grabbit/tests/data/7t_trt/sub-05/ses-2/func/sub-05_ses-2_task-rest_acq-prefrontal_physio.tsv.gz']

## Retrieve first run BOLD images for all subjects

In [38]:
layout.get(subject=1)[:2]
set(x.type for x in layout.get(subject=1))

{'T1map',
 'T1w',
 'bold',
 'magnitude1',
 'magnitude2',
 'phasediff',
 'physio',
 'scans',
 'sessions'}

In [39]:
hits = layout.get(run=1, type='bold')
assert hits, "We should have got some hits"  # sanity check
print("Each hit is a named tuple, e.g.:\n\t%s" % repr(hits[0]))

print("\nFirst 10 filenames:\n\t%s" % 
    '\n\t'.join([h.filename for h in hits][:10]))

Each hit is a named tuple, e.g.:
	File(filename='../grabbit/tests/data/7t_trt/sub-01/ses-1/func/sub-01_ses-1_task-rest_acq-fullbrain_run-1_bold.nii.gz', task='rest_acq', run='1', session='1', type='bold', acquisition='fullbrain_run', subject='01')

First 10 filenames:
	../grabbit/tests/data/7t_trt/sub-01/ses-1/func/sub-01_ses-1_task-rest_acq-fullbrain_run-1_bold.nii.gz
	../grabbit/tests/data/7t_trt/sub-01/ses-2/func/sub-01_ses-2_task-rest_acq-fullbrain_run-1_bold.nii.gz
	../grabbit/tests/data/7t_trt/sub-02/ses-1/func/sub-02_ses-1_task-rest_acq-fullbrain_run-1_bold.nii.gz
	../grabbit/tests/data/7t_trt/sub-02/ses-2/func/sub-02_ses-2_task-rest_acq-fullbrain_run-1_bold.nii.gz
	../grabbit/tests/data/7t_trt/sub-03/ses-1/func/sub-03_ses-1_task-rest_acq-fullbrain_run-1_bold.nii.gz
	../grabbit/tests/data/7t_trt/sub-03/ses-2/func/sub-03_ses-2_task-rest_acq-fullbrain_run-1_bold.nii.gz
	../grabbit/tests/data/7t_trt/sub-04/ses-1/func/sub-04_ses-1_task-rest_acq-fullbrain_run-1_bold.nii.gz
	../grabbi

## Get all unique subject IDs

In [40]:
layout.get(return_type='id', target='subject')

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']